In [30]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [31]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [32]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [33]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [34]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  beta_regu = tf.placeholder(tf.float32)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))+beta_regu*tf.nn.l2_loss(weights)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [35]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,beta_regu : 1e-3}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))


Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 22.871023
Minibatch accuracy: 3.9%
Validation accuracy: 8.8%
Minibatch loss at step 500: 2.508278
Minibatch accuracy: 78.9%
Validation accuracy: 76.1%
Minibatch loss at step 1000: 1.758787
Minibatch accuracy: 81.2%
Validation accuracy: 78.2%
Minibatch loss at step 1500: 0.970301
Minibatch accuracy: 84.4%
Validation accuracy: 79.8%
Minibatch loss at step 2000: 0.879148
Minibatch accuracy: 82.0%
Validation accuracy: 80.9%
Minibatch loss at step 2500: 0.740427
Minibatch accuracy: 80.5%
Validation accuracy: 81.6%
Minibatch loss at step 3000: 0.795191
Minibatch accuracy: 81.2%
Validation accuracy: 81.9%
Test accuracy: 88.8%


In [36]:
#SGD with relu
batch_size = 128
relu_count = 1024 #hidden nodes count

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  beta_regu = tf.placeholder(tf.float32)
  

    # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, relu_count]))
  biases_1 = tf.Variable(tf.zeros([relu_count]))
  # send relu to final nn layer
  weights_2 = tf.Variable(
    tf.truncated_normal([relu_count, num_labels]))

  biases_2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation. (#layer_1 -> layer_2(relu) -> layer_3)
  logits = tf.matmul( tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1), weights_2) + biases_2


  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=tf_train_labels))+beta_regu*(tf.nn.l2_loss(weights_1)+tf.nn.l2_loss(weights_2))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul( tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) + biases_2)
  test_prediction = tf.nn.softmax(
        tf.matmul( tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) + biases_2)

In [37]:
num_steps = 5000

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,beta_regu : 1e-3}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 686.929810
Minibatch accuracy: 10.2%
Validation accuracy: 28.7%
Minibatch loss at step 500: 194.724045
Minibatch accuracy: 83.6%
Validation accuracy: 79.7%
Minibatch loss at step 1000: 115.424736
Minibatch accuracy: 80.5%
Validation accuracy: 81.9%
Minibatch loss at step 1500: 68.269287
Minibatch accuracy: 87.5%
Validation accuracy: 83.8%
Minibatch loss at step 2000: 41.187836
Minibatch accuracy: 89.1%
Validation accuracy: 85.0%
Minibatch loss at step 2500: 25.069437
Minibatch accuracy: 90.6%
Validation accuracy: 85.7%
Minibatch loss at step 3000: 15.471210
Minibatch accuracy: 87.5%
Validation accuracy: 86.8%
Minibatch loss at step 3500: 9.576359
Minibatch accuracy: 89.1%
Validation accuracy: 87.2%
Minibatch loss at step 4000: 6.026484
Minibatch accuracy: 89.8%
Validation accuracy: 87.9%
Minibatch loss at step 4500: 3.767484
Minibatch accuracy: 91.4%
Validation accuracy: 87.3

In [38]:
#SGD with relu
batch_size = 128
relu_count = 1024 #hidden nodes count

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  beta_regu = tf.placeholder(tf.float32)
  

    # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, relu_count]))
  biases_1 = tf.Variable(tf.zeros([relu_count]))
  # send relu to final nn layer
  weights_2 = tf.Variable(
    tf.truncated_normal([relu_count, num_labels]))

  biases_2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation. (#layer_1 -> layer_2(relu) -> layer_3)
  logits = tf.matmul( tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1), weights_2) + biases_2


  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul( tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) + biases_2)
  test_prediction = tf.nn.softmax(
        tf.matmul( tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) + biases_2)

In [39]:
num_steps = 101

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,beta_regu : 1e-3}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 2 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 307.059570
Minibatch accuracy: 10.2%
Validation accuracy: 32.5%
Minibatch loss at step 2: 1270.988892
Minibatch accuracy: 34.4%
Validation accuracy: 34.4%
Minibatch loss at step 4: 540.280151
Minibatch accuracy: 39.8%
Validation accuracy: 45.1%
Minibatch loss at step 6: 329.107788
Minibatch accuracy: 64.8%
Validation accuracy: 66.7%
Minibatch loss at step 8: 240.280273
Minibatch accuracy: 67.2%
Validation accuracy: 67.7%
Minibatch loss at step 10: 174.630127
Minibatch accuracy: 75.0%
Validation accuracy: 71.6%
Minibatch loss at step 12: 58.579941
Minibatch accuracy: 77.3%
Validation accuracy: 77.2%
Minibatch loss at step 14: 115.400116
Minibatch accuracy: 72.7%
Validation accuracy: 74.7%
Minibatch loss at step 16: 143.093445
Minibatch accuracy: 68.0%
Validation accuracy: 72.6%
Minibatch loss at step 18: 169.502747
Minibatch accuracy: 75.0%
Validation accuracy: 77.5%
Minibatch

In [40]:
#SGD with relu
batch_size = 128
relu_count = 1024 #hidden nodes count

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  

    # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, relu_count]))
  biases_1 = tf.Variable(tf.zeros([relu_count]))
  # send relu to final nn layer
  weights_2 = tf.Variable(
    tf.truncated_normal([relu_count, num_labels]))

  biases_2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation. (#layer_1 -> layer_2(relu) -> layer_3)
  logits = tf.matmul(tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1),0.5), weights_2) + biases_2


  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) + biases_2)
  test_prediction = tf.nn.softmax(
        tf.matmul( tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) + biases_2)

In [41]:
num_steps = 101

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 2 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 539.154297
Minibatch accuracy: 14.8%
Validation accuracy: 37.4%
Minibatch loss at step 2: 1508.225586
Minibatch accuracy: 27.3%
Validation accuracy: 29.3%
Minibatch loss at step 4: 496.077576
Minibatch accuracy: 42.2%
Validation accuracy: 55.6%
Minibatch loss at step 6: 166.990372
Minibatch accuracy: 62.5%
Validation accuracy: 63.0%
Minibatch loss at step 8: 438.092224
Minibatch accuracy: 53.1%
Validation accuracy: 68.3%
Minibatch loss at step 10: 183.081772
Minibatch accuracy: 71.1%
Validation accuracy: 75.5%
Minibatch loss at step 12: 84.174454
Minibatch accuracy: 74.2%
Validation accuracy: 74.6%
Minibatch loss at step 14: 167.856812
Minibatch accuracy: 67.2%
Validation accuracy: 74.2%
Minibatch loss at step 16: 148.195251
Minibatch accuracy: 68.0%
Validation accuracy: 76.8%
Minibatch loss at step 18: 89.111320
Minibatch accuracy: 78.1%
Validation accuracy: 75.8%
Minibatch 

In [54]:
#SGD with relu
batch_size = 128
h1_layer_nodes = 1024
h2_layer_nodes = 1024
h3_layer_nodes = 1024
beta_regu = 1e-3

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  global_step = tf.Variable(0)

    # Variables.
  weights_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, h1_layer_nodes],stddev = np.sqrt(2.0/(image_size*image_size))))
  biases_1 = tf.Variable(tf.zeros([h1_layer_nodes]))
  # send relu to final nn layer
  weights_2 = tf.Variable(
    tf.truncated_normal([h1_layer_nodes, h2_layer_nodes],stddev = np.sqrt(2.0/(h1_layer_nodes))))

  biases_2 = tf.Variable(tf.zeros([h2_layer_nodes]))
  weights_3 = tf.Variable(tf.truncated_normal([h2_layer_nodes,h3_layer_nodes],stddev = np.sqrt(2.0/(h2_layer_nodes))))
  biases_3 = tf.Variable(tf.zeros([h3_layer_nodes]))
  weights_4 = tf.Variable(tf.truncated_normal([h3_layer_nodes,num_labels],stddev = np.sqrt(2.0/h3_layer_nodes)))
  biases_4 = tf.Variable(tf.zeros([num_labels]))
  
  relu1_layer = tf.nn.relu(tf.matmul(tf_train_dataset,weights_1)+biases_1)
  relu2_layer = tf.nn.relu(tf.matmul(relu1_layer,weights_2)+biases_2)
  relu3_layer = tf.nn.relu(tf.matmul(relu2_layer,weights_3)+biases_3)
  logits = tf.matmul(relu3_layer,weights_4)+biases_4

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=tf_train_labels))+beta_regu*(tf.nn.l2_loss(weights_1)+tf.nn.l2_loss(weights_2)+tf.nn.l2_loss(weights_3)+tf.nn.l2_loss(weights_4))
  
  # Optimizer.
  learning_rate = tf.train.exponential_decay(0.5,global_step,1000,0.65,staircase = True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step =global_step)
  
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  relu1_layer = tf.nn.relu(tf.matmul(tf_valid_dataset,weights_1)+biases_1)
  relu2_layer = tf.nn.relu(tf.matmul(relu1_layer,weights_2)+biases_2)
  valid_prediction = tf.nn.softmax(tf.matmul(relu2_layer,weights_3)+biases_3)
  relu1_layer = tf.nn.relu(tf.matmul(tf_test_dataset,weights_1)+biases_1)
  relu2_layer = tf.nn.relu(tf.matmul(relu1_layer,weights_2)+biases_2)  
  test_prediction = tf.nn.softmax(tf.matmul(relu2_layer,weights_3) + biases_3)

In [ ]:
num_steps = 9001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 4.769846
Minibatch accuracy: 6.2%
Validation accuracy: 0.0%
Minibatch loss at step 500: 1.892788
Minibatch accuracy: 85.9%
Validation accuracy: 0.0%
